【本周思路】本周通过进一步学习递归算法，引入动态规划的知识
* 408 引入分治策略的概念
* 409 介绍优化问题，和解决优化问题的贪心策略
* 410 找零问题【递归版本】
* 411 找零问题【动态规划版本】
* 412 动态规划的进一步案例
* 413 这两周的总结

**Table of contents**<a id='toc0_'></a>    
- [408 分治策略](#toc1_)    
- [优化问题和贪心策略](#toc2_)    
- [递归解法解决找零问题](#toc3_)    
- [动态规划](#toc4_)    
- [博物馆大盗问题](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[408 分治策略](#toc0_)
## 1. 分治策略：分而治之
当面临一个又大又复杂的问题，可以采用这个策略：
* 将问题分为若干更小规模的部分
* 通过解决每个小规模部分问题，并将结果汇总得到原问题的解

即：

![分治策略](./img/408.PNG)

## 2. 递归算法与分治策略

* 递归算法是分治策略的一种【递归算法需要调用自身】，具体体现有：
    * 问题解决依赖于若干缩小了规模的问题
    * 汇总得到原问题的解
* 分治策略有广泛的应用【而且大多都是用递归算法来解决的】：
    * 排序
    * 查找
    * 遍历
    * 求值等

# <a id='toc2_'></a>[409 优化问题和贪心策略](#toc0_)
这一节重点：
* 什么是优化问题？
* 什么是贪心策略？

## 1. 优化问题
计算机科学中许多算法都是为了找到某些问题的最优解，典型问题就是**兑换最少个数的硬币**:

假设你为一家自动售货机厂家编程序，要求自动售货机要每次找给顾客最少数量硬币

例如某次顾客投进$1纸币,买了ȼ37的东西,要找ȼ63,那么最少数量就是:2个quarter(ȼ25)，1个dime(ȼ10)和3个penny(ȼ1)，一共6枚

怎么算出来的？

$\implies$

## 2. 贪心策略【每次都试图解决问题的尽量大的一部分】

* 从最大面值的硬币开始，用尽量多的最大面值
* 有余额的，再到下一最大面值的硬币，还用尽量多的这一面值，一直到penny（ȼ1）为止

也就是每次以**最**多数量的**最**大面值硬币来**最**快减少找零面值

但是！！这一策略在特殊情况下会失效

$\implies$每一步都是最优的，并不代表结果是最优的

补充：

贪心策略能保证得到最优解的条件（需要同时满足）：
* 贪心选择性质： 问题的全局最优解可以通过一系列局部最优（贪心）选择来达到。这意味着做出局部最优选择后，剩下的子问题可以独立求解，且这个选择不会被后面的选择推翻
* 最优子结构： 问题的最优解包含了其子问题的最优解。也就是说，可以通过子问题的最优解来构造原问题的最优解（这是动态规划也需要的性质）

如果一个问题只满足最优子结构，但不满足贪心选择性质，那么贪心策略不能保证得到最优解（可能得到次优解），通常需要使用动态规划等其他方法（例如经典的0-1背包问题）

In [ ]:
lis = [1,5,20,50]
def change(lis,num):
    changelis = []
    for i in lis[::-1]:
        a = num // i
        num = num % i
        changelis.append(a)
    return changelis
print(change(lis,63))

[1, 0, 2, 3]


# <a id='toc3_'></a>[递归解法解决找零问题](#toc0_)
1. 最小规模：只有一枚硬币
2. 减小规模：找零减去对应的硬币零钱，求兑换数量【调用自身】


In [4]:
def recMC(coinValueList,change):#重复计算极其低效
    minCoins = change
    if change in coinValueList:
        minCoins = 1
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1+ recMC(coinValueList,change - i)
            if numCoins < minCoins:
                minCoins = numCoins
    return minCoins
print(recMC([1,5,10,25],63))

6


In [ ]:
# 递归改进
def recMC(coinValueList,change,knownResult):
    minCoins = change
    if change in coinValueList:
        knownResult[change] = 1#结果记录
        return 1
    elif knownResult[change] > 0:
        return knownResult[change]
    else:
        for i in [c for c in coinValueList if c <= change]:
            numCoins = 1+ recMC(coinValueList,change - i,knownResult)
            if numCoins < minCoins:
                minCoins = numCoins
                knownResult[change] = minCoins
    return minCoins
print(recMC([1,5,10,25],65,[0]*66))

4


# <a id='toc4_'></a>[动态规划](#toc0_)
1. 动态规划算法采用了一种更有条理的方式来得到问题的解
* 找零兑换的动态规划算法从最简单的“1分钱找零”的最优解开始，逐步递加上去，直到我们需要的找零钱数
* 在找零递加的过程中，设法保持每一分钱的递加都是最优解，一直加到求解找零钱数，自然得到最优解
* 递加的过程能保持最优解的关键是，其依赖于更少钱数最优解的简单计算，而更少钱数的最优解已经得到了。
* 问题的最优解包含了更小规模子问题的最优解，这是一个最优化问题能够用动态规划策略解决的必要条件。
2. 建立动态规划的表格
![图片](./img/1.png)

In [ ]:
def dpMakeChange(coinValueList,change,minCoins):
    for cents in range(1,change + 1):
        coinCount = cents

        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents - j] + 1 < coinCount:
                coinCount = minCoins[cents -j] + 1
        minCoins[cents] = coinCount
    return minCoins[change]
print(dpMakeChange([1,5,10,21,25],63,[0]*64))

3


# <a id='toc5_'></a>[博物馆大盗问题](#toc0_)
![图片](./img/2.png)

1. 贪心策略明显不行了，只能使用动态规划
![pic](./img/3.png)

2.  列出来表格有
![pic](./img/4.png)

3. 每个格子的内容，都只依赖于上边和左边

In [10]:
#dp
#without function
tr = [None,{'w':2,'v':3},{'w':3,'v':4},{'w':4,'v':8},{'w':5,'v':8},{'w':9,'v':10}]

max_w = 20

m = {(i,w):0 for i in range(len(tr))
                for w in range(max_w + 1)}
#print(m)
for i in range(1,len(tr)):
    for w in range(1,max_w + 1):
        if tr[i]['w'] > w:
            m[(i,w)] = m[(i-1,w)]
        else:
            m[(i,w)] = max(
                m[(i-1,w)],
                m[(i-1,w-tr[i]['w'])] + tr[i]['v']
            )

print(m[(len(tr)-1,max_w)])

29


In [ ]:
#recursive
tr = {(2,3),(3,4),(4,8),(5,8),(9,10)}
max_w = 20

m = {}

def thief(tr,w):
    if tr == set() or w == 0:
        m[(tuple(tr),w)] = 0
        return 0
    elif (tuple(tr),w) in m:
        return m[(tuple(tr),w)]
    else:
        vmax = 0
        for t in tr:
            if t[0] <= w:
                v = thief(tr - {t},w - t[0]) + t[1]
                vmax = max(vmax,v)
        m[(tuple(tr),w)] = vmax
        return vmax
print(thief(tr,max_w))